# Indivudal User Stats

## Imports

In [33]:
import pandas as pd
import plotly.express as px
import re
import ipywidgets as widgets

## Import CSV

In [34]:
df = pd.read_csv('./extraction/emissions.csv')
df.head()

,address,controller,action,endpoint,new_year_resolution,discipline,username,status,items/total,red_meat_co2_emissions,...,tv_hours,computer_co2_emissions,computer_hours,car_co2_emissions,car_hours,walking_co2_emissions,walking_hours,public_transport_co2_emissions,public_transport_hours,tv_hour
0,178373,LocationController,Get,(OWL) https://uwo.ca/edge/software,5%,software,Sammy,4,2/1,362.873600,...,43.0,546.0,130,975.0,150,0,3,43.0,10,NaN
1,156354,Controlmite,Get,(OWL) https://uwo.ca/edge/software,15%,software,Griffin,0,3/1,322.957504,...,75.0,369.6,88,487.5,75,0,68,184.9,43,NaN
2,919323,ControlPublic,Get,https://uwo.ca/edge/electrical,9%,electrical,Spencer,0,3/5,94.347136,...,50.0,315.0,75,546.0,84,0,65,335.4,78,NaN
3,983575,Controlz,Get,(OWL) https://uwo.ca/edge/mechanical,4%,mechanical,Annie,0,3/1,127.005760,...,50.0,21.0,5,130.0,20,0,71,146.2,34,NaN
4,919325,LocationController,Get,https://uwo.ca/edge/mechanical,6%,mechanical,Jessica,0,3/3,199.580480,...,100.0,210.0,50,507.0,78,0,101,94.6,22,NaN


## Graph

### Graph Data

In [35]:
# List of tracked columns
emission_columns = [
    'red_meat_co2_emissions',
    'grains_co2_emissions',
    'dairy_co2_emissions',
    'cellphone_co2_emissions',
    'tv_co2_emissions', 
    'computer_co2_emissions',
    'car_co2_emissions',
    'public_transport_co2_emissions'
]

In [36]:
# Get user info
username = input("Please enter your username: ")
values = df.loc[df['username'] == username]
user_values = [values.iloc[0][k] for k in emission_columns] # Get the values of the column

# create a more accessable map
user_value_map = {"red_meat": user_values[0], "grains": user_values[1], "dairy": user_values[2], "phone": user_values[3], "tv": user_values[4], "computer": user_values[5], "car": user_values[6], "public_transport": user_values[7]}

### Plotting the Graph

In [37]:
# print the bar chart dynamically
def build_graph (data):
    fig = px.bar(
        x=data.keys(), # formatting the columns
        y=data.values(),
        title=f"{username}'s CO2 emissions from the past year", # appl the title
        labels={ # apply labels
            "x": "form of emission", 
            "y": "kgs of CO2",
        },
    )
    fig.update_xaxes(categoryorder="total ascending") # convert axaxes to be ascending
    fig.show()

### Sliders to Update Graph

In [42]:
def create_slider(description:str, value:float, min_val, max_val, step:float)->widgets.FloatSlider:
    return widgets.FloatSlider(
        value=value,
        min=min_val,
        max=max_val,
        step=step,
        description=description,
        layout={"width": "1000px"}
    )

red_meat_slider = create_slider("red_meat", user_value_map["red_meat"], user_value_map["red_meat"]/1.5, user_value_map["red_meat"]*1.5, 10)
grains_slider = create_slider("grains", user_value_map["grains"], user_value_map["grains"]/1.5, user_value_map["grains"]*1.5, 0.5)
dairy_slider = create_slider("dairy", user_value_map["dairy"], user_value_map["dairy"]/1.5, user_value_map["dairy"]*1.5, 10)
phone_slider = create_slider("phone", user_value_map["phone"], user_value_map["phone"]/1.5, user_value_map["phone"]*1.5, 10)
tv_slider = create_slider("tv", user_value_map["tv"], user_value_map["tv"]/1.5, user_value_map["tv"]*1.5, 10)
car_slider = create_slider("car", user_value_map["car"], user_value_map["car"]/1.5, user_value_map["car"]*1.5, 10)
public_transport_slider = create_slider("public_transport", user_value_map["public_transport"], user_value_map["public_transport"]/1.5, user_value_map["public_transport"]*1.5, 5)
computer_slider = create_slider("computer", user_value_map["computer"], user_value_map["computer"]/1.5, user_value_map["computer"]*1.5, 10)


sliders = [red_meat_slider, grains_slider, dairy_slider, phone_slider, tv_slider, computer_slider, car_slider, public_transport_slider]

total = sum([slider.value for slider in sliders])
changed_total = total

changesTextBox = widgets.Text(
    value=f"New Years Resolution Change: {(changed_total/total-1)*100}%",
    disabled=True,
    layout={"width": "1000px"}
)

initialValues = widgets.Text(
    value=f"Car {6.5}, Public Transport {4.3}",
    disabled=True,
    layout={"width": "1000px"}
)

updateGraphBtn = widgets.Button(
    description="Update Graph",
)

def on_slider_change(change):
    global changed_total
    diff = change['new'] - change['old']
    changed_total += diff
    changesTextBox.value = f"New Years Resolution Change: {(changed_total/total-1)*100}%"


def graphUpdate(btn):
    for slider in sliders:
        user_value_map[slider.description] = slider.value
    build_graph(user_value_map)

updateGraphBtn.on_click(graphUpdate)


for slider in sliders:
    slider.observe(on_slider_change, 'value')

all_widgets = sliders + [changesTextBox, initialValues, updateGraphBtn]

build_graph(user_value_map)
widgets.VBox(
    all_widgets,
)
